## Set up Packages

In [13]:
import numpy as np
import pandas as pd
from transformers import BertTokenizer, Trainer, BertForSequenceClassification, TrainingArguments, logging
from datasets import Dataset, load_dataset
import torch
import logging
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

## Load Model and Tokenizer

In [14]:
model = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-pretrain',num_labels=3)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-pretrain')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at yiyanghkust/finbert-pretrain and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Load up Dataset

In [15]:
dataset = load_dataset("financial_phrasebank", "sentences_allagree")["train"]
print(dataset)

x = dataset["sentence"]
y = dataset["label"]

x_train, x_temp, y_train,y_temp = train_test_split(x,y,test_size=0.3, random_state=42, shuffle=True)
x_test, x_eval, y_test, y_eval = train_test_split(x_temp, y_temp,test_size = 0.5, random_state = 42, shuffle = True)

train_df = pd.DataFrame({
    "sentence": x_train,
    "label": y_train
})

test_df = pd.DataFrame({
    "sentence": x_test,
    "label": y_test
})

eval_df = pd.DataFrame({
    "sentence": x_eval,
    "label": x_eval
})


print(train_df.shape)
print(test_df.shape)
print(eval_df.shape)

train_ds = Dataset.from_pandas(train_df)
test_ds = Dataset.from_pandas(test_df)
eval_ds = Dataset.from_pandas(eval_df)

train_ds = train_ds.map(lambda x: tokenizer(x["sentence"],truncation = True, padding = "max_length", max_length = 256), batched = True)
test_ds = train_ds.map(lambda x: tokenizer(x["sentence"],truncation = True, padding = "max_length", max_length = 256), batched = True)
eval_ds = train_ds.map(lambda x: tokenizer(x["sentence"],truncation = True, padding = "max_length", max_length = 256), batched = True)

print(train_ds)


Dataset({
    features: ['sentence', 'label'],
    num_rows: 2264
})
(1584, 2)
(340, 2)
(340, 2)


Map:   0%|          | 0/1584 [00:00<?, ? examples/s]

Map:   0%|          | 0/1584 [00:00<?, ? examples/s]

Map:   0%|          | 0/1584 [00:00<?, ? examples/s]

Dataset({
    features: ['sentence', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 1584
})


## Evaluation Function and Argument Set Up

In [19]:
def evaluation(datapoint):
    predictions, labels = datapoint
    predictions = np.argmax(predictions,axis = 1)
    return {'accuracy': accuracy_score(predictions, labels)}

args = TrainingArguments(
    logging_dir='./logs',
    output_dir = 'temp/',
    logging_strategy='epoch',
    evaluation_strategy = 'epoch',
    save_strategy = 'epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
)
trainer = Trainer(
    model=model,                         
    args=args,                  
    train_dataset=train_ds,        
    eval_dataset=eval_ds,            
    compute_metrics=evaluation
)

## Run the model before the rest

In [20]:
model.eval()
trainer.predict(test_ds).metrics

{'test_loss': 1.1328144073486328,
 'test_accuracy': 0.2588383838383838,
 'test_runtime': 6.4235,
 'test_samples_per_second': 246.595,
 'test_steps_per_second': 7.784}

## Model Training

In [21]:
trainer.train()

wandb: Currently logged in as: kensh1nhualuoweiyang (kensh1nhualuoweiyan). Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy
1,0.548400,0.181993,0.950126
2,0.135700,0.045543,0.988005
3,0.046800,0.017655,0.995581
4,0.023600,0.010628,0.997475
5,0.012200,0.008281,0.998106


TrainOutput(global_step=250, training_loss=0.15334275531768798, metrics={'train_runtime': 137.4501, 'train_samples_per_second': 57.621, 'train_steps_per_second': 1.819, 'total_flos': 1041929134202880.0, 'train_loss': 0.15334275531768798, 'epoch': 5.0})

## Complete and Save The Model

In [22]:
trainer.save_model('finbertNLP/')

## Link to WandB
https://wandb.ai/kensh1nhualuoweiyan/huggingface/reports/Fin-Bert-Fin-Tunning--Vmlldzo3MzQ5NjQ5